# Automated Hamiltonian Extract


## Imports and file structure creation

In [1]:
import os
from qiskit_metal import MetalGUI

from validate_config_file import *
from helpers import *
from make_designs import *
from prepare_inductex_gds_files import *
from prepare_palace_sims import *
from prepare_inductex_sims import *
from run_simulations import *
from collect_simulation_results import *

# MPI related environment variables 
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ["PMIX_MCA_gds"]="hash"

gds_types                       = ['raw', 'processed_multi_layer', 'processed_single_layer', 'simulations']
design_names                    = ['full', 'full_no_jj', 'junction', 'transmon', 'transmon_no_jj', 'resonator', 'feedline']
inductex_simulation_types       = ['capacitance', 'inductance', 'sparams', 'electric_field', 'magnetic_field']
palace_simulation_types         = ['capacitance', 'eigenmode']
os.makedirs('data', exist_ok=True)
for gds_type in gds_types:
    os.makedirs(f'gds_files/{gds_type}', exist_ok=True)

for design in design_names:
    for sim_type in inductex_simulation_types:
        os.makedirs(f'simulations/inductex/{sim_type}/{design}', exist_ok=True)


for design in design_names:
    for sim_type in palace_simulation_types:
        os.makedirs(f'simulations/palace/{sim_type}/{design}', exist_ok=True)


# TODO
# Add suport for none entries in config file
# Double check chip z height is correct
# Find a way to remove the xmon dummy
#   Prehaps this can bedone with replace junction
# Cell hirachy might be a bit busted for GDS files
# Multiple declarations of component geos, check helper functions for uncertainty
# claw_cpw_width and calw_cpw_length dont seem to do anything
# The json file for all the geometry data is slightly contradictory as geometry for some things have been declared 2 or 3 times
# this is most likely due to different simulations being done on indvidiual components or a partial design of the system
# Removed end_straight=resonator_geo['end_straight'] from resonator as it was none and causing issues
# Bad implementation of pin_width = pin_width*1e-3 in _get_pin_left_um and _get_pin_right_um
# Force adjusted CPW for transmon to fit full design, didnt work use in report for source of error
# Feed cap simulation uses feedline width to determine where to place ground label, but in reality thickness of feedline is done using resonator width couldn't
# cause problems unless feedline width or resonator width is REALLY extreme
# Sparam for transmon is not quite right, will fix later
# Rename data to ref-design-data?
# Chip thickness for make_designs might be too thin
# Meshing problem with feedline cap sim
# Sparams are pasted in weirdly
# Slight bug with palace saying succesful or unsucessful when keep_open is true
# Way too much effort to get transmon eigen sim working, don't need it since we can just use the full, will just cause problems

## Load and validate setup file

In [2]:
CONFIG_PATH                     = "quantum_config.json"
quantum_config                  = ensure_config(CONFIG_PATH)
run_inductex                    = False
run_palace_setup                = False
run_palace                      = False
palace_show_conductor_indices   = False

[config] Validation successful: 'quantum_config.json' is valid.
[INFO] simulations"]["inductex"]["ixi"]["sparams_steps is: 8 (Default: 200)
[INFO] simulations"]["palace"]["eigenmode"]["fine_mesh_min_size_components is: 5e-06 (Default: 1e-05)
[INFO] simulations"]["palace"]["eigenmode"]["fine_mesh_min_size_junction is: 1e-05 (Default: 2e-06)


## Find design from database and get master design file


In [3]:
design_specs                                 = quantum_config["design_specs"]

if os.path.exists("data/design_reference_data_all.json"):
    best_design, ref_design_specs, ref_capacitance_specs, qubit_geo, resonator_geo, feedline_geo, lj = fetch_design_fast()
else:
    best_design, ref_design_specs, ref_capacitance_specs, qubit_geo, resonator_geo, feedline_geo, lj = fetch_design(design_specs=design_specs, num_top=1)

## Save master design file and useful data

In [4]:
save_results_to_json(best_design, name="data/design_reference_data_all", save=True)
save_results_to_json(ref_design_specs, name="data/design_reference_data_design_specs", save=True)
save_results_to_json(ref_capacitance_specs, name="data/design_reference_data_capacitance", save=True)
save_results_to_json(qubit_geo, name="data/design_reference_qubit_geometry", save=True)
save_results_to_json(resonator_geo, name="data/design_reference_resonator_geometry", save=True)
save_results_to_json(feedline_geo, name="data/design_reference_feedline_geometry", save=True)
save_results_to_json(lj, name="data/design_reference_junction_inductance", save=True)

✅ Saved to data/design_reference_data_all.json
✅ Saved to data/design_reference_data_design_specs.json
✅ Saved to data/design_reference_data_capacitance.json
✅ Saved to data/design_reference_qubit_geometry.json
✅ Saved to data/design_reference_resonator_geometry.json
✅ Saved to data/design_reference_feedline_geometry.json
✅ Saved to data/design_reference_junction_inductance.json


## Generate Qiskit Metal designs from master design geomtery and config file

In [5]:
scaling_factor              = quantum_config["miscellaneous"]["sims"]["base_chip_scaling_factor"]
full_design                 = make_full_design(best_design, qubit_geo, resonator_geo, feedline_geo)
full_no_jj_design           = make_full_no_jj_design(best_design, qubit_geo, resonator_geo, feedline_geo)
junction_design             = make_junction_design(best_design, qubit_geo, resonator_geo, feedline_geo, scaling_factor=scaling_factor)
transmon_design             = make_transmon_design(best_design, qubit_geo, resonator_geo, feedline_geo, scaling_factor=scaling_factor)
transmon_no_jj_design       = make_transmon_no_jj_design(best_design, qubit_geo, resonator_geo, feedline_geo, scaling_factor=scaling_factor)
resonator_design            = make_resonator_design(best_design, qubit_geo, resonator_geo, feedline_geo, scaling_factor=scaling_factor)
feedline_design             = make_feedline_design(best_design, qubit_geo, resonator_geo, feedline_geo, scaling_factor=scaling_factor)

## Save raw designs to GDS

In [6]:
to_gds(full_design, filename="gds_files/raw/full.gds")  
to_gds(full_no_jj_design, filename="gds_files/raw/full_no_jj.gds")  
to_gds(junction_design, filename="gds_files/raw/junction.gds")  
to_gds(transmon_design, filename="gds_files/raw/transmon.gds")  
to_gds(transmon_no_jj_design, filename="gds_files/raw/transmon_no_jj.gds")  
to_gds(resonator_design, filename="gds_files/raw/resonator.gds")  
to_gds(feedline_design, filename="gds_files/raw/feedline.gds")  

08:41PM 16s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
08:41PM 16s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"c:\Users\reece\Desktop\Final Year Work\2nd Semester\Project (E) 448\Simulations\resources".
08:41PM 16s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to replace junction. Checked directory:"c:\Users\reece\Desktop\Final Year Work\2nd Semester\Project (E) 448\Simulations\resources".
08:41PM 16s WARNING [_qgeometry_to_gds]: Unexpected shapely object geometry.The variable qgeometry_element is <class 'numpy.float64'>, method can currently handle Polygon and FlexPath.
08:41PM 16s WARNING [_import_junctions_to_one_cell]: Not able to find file:"../resources/Fake_Junctions.GDS".  Not used to rep

## Process InductEx GDS files

In [7]:
process_full_gds()
process_full_no_jj_gds()
process_junction_gds()
process_transmon_gds()
process_transmon_no_jj_gds()
process_resonator_gds()
process_feedline_gds()

Boolean AND((1, 0), (3, 0)) → (0, 0) complete.
Boolean AND((0, 0), (4, 0)) → (0, 0) complete.
Boolean OR((4, 10), (4, 11)) → (4, 10) complete.
Deleted layer 1, datatype 0
Deleted layer 3, datatype 0
Deleted layer 4, datatype 0
Deleted layer 4, datatype 11
✅  Wrote gds_files/processed_multi_layer/full_processed_multi.gds
Boolean OR((0, 0), (1, 0)) → (0, 0) complete.
Boolean OR((0, 0), (2, 0)) → (0, 0) complete.
Boolean OR((0, 0), (3, 0)) → (0, 0) complete.
Boolean OR((0, 0), (4, 0)) → (0, 0) complete.
Deleted layer 1, datatype 0
Deleted layer 2, datatype 0
Deleted layer 3, datatype 0
Deleted layer 4, datatype 0
✅  Wrote gds_files/processed_single_layer/full_processed.gds
Deleted layer 2, datatype 0
✅  Wrote gds_files/processed_multi_layer/full_no_jj_processed_multi.gds
Boolean OR((0, 0), (1, 0)) → (0, 0) complete.
Boolean OR((0, 0), (3, 0)) → (0, 0) complete.
Boolean OR((0, 0), (4, 0)) → (0, 0) complete.
Deleted layer 1, datatype 0
Deleted layer 3, datatype 0
Deleted layer 4, datatype 0

## Setup InductEx GDS simulation files

In [8]:
# Capacitance Sim GDS Files
make_inductex_cap_sim_transmon_gds(transmon_design, qubit_geo)              
make_inductex_cap_sim_transmon_no_jj_gds(transmon_no_jj_design, qubit_geo)  
make_inductex_cap_sim_resonator_gds(resonator_design, resonator_geo)        
make_inductex_cap_sim_feedline_gds(feedline_design, feedline_geo)           

# Inductance Sim GDS Files
make_inductex_ind_sim_resonator_gds(resonator_design, resonator_geo)        

# S-Parameter Sim GDS Files
make_inductex_sparam_sim_transmon_gds(transmon_design, qubit_geo)                   
make_inductex_sparam_sim_transmon_no_jj_gds(transmon_no_jj_design, qubit_geo)       
make_inductex_sparam_sim_resonator_gds(resonator_design, resonator_geo)             

✅  Wrote simulations/inductex/capacitance/transmon/transmon_cap.gds
✅  Wrote simulations/inductex/capacitance/transmon_no_jj/transmon_no_jj_cap.gds
✅  Wrote simulations/inductex/capacitance/resonator/resonator_cap.gds
✅  Wrote simulations/inductex/capacitance/feedline/feedline_cap.gds
✅  Wrote simulations/inductex/inductance/resonator/resonator_ind.gds
✅  Wrote simulations/inductex/sparams/transmon/transmon_sparams.gds
✅  Wrote simulations/inductex/sparams/transmon_no_jj/transmon_no_jj_sparams.gds
✅  Wrote simulations/inductex/sparams/resonator/resonator_sparams.gds


In [9]:
# UNIMPLEMENTED INDUCTEX GDS SIM FILES

#make_inductex_cap_sim_full_gds(full_design, qubit_geo)
#make_inductex_cap_sim_full_no_jj_gds(full_no_jj_design, qubit_geo)
#make_inductex_cap_sim_junction_gds(junction_design, qubit_geo)

#make_inductex_ind_sim_full_gds(full_design, qubit_geo)
#make_inductex_ind_sim_full_no_jj_gds(full_no_jj_design, qubit_geo)
#make_inductex_ind_sim_junction_gds(junction_design, qubit_geo)
#make_inductex_ind_sim_transmon_gds(transmon_design, qubit_geo)
#make_inductex_ind_sim_transmon_no_jj_gds(transmon_no_jj_design, qubit_geo)
#make_inductex_ind_sim_feedline_gds(feedline_design, feedline_geo)

#make_inductex_sparam_sim_full_gds(full_design, qubit_geo)
#make_inductex_sparam_sim_full_no_jj_gds(full_no_jj_design, qubit_geo)
#make_inductex_sparam_sim_junction_gds(junction_design, qubit_geo)
#make_inductex_sparam_sim_feedline_gds(feedline_design, feedline_geo)

## Setup InductEx IXI, LDF and CIR simulation files

In [10]:
quantum_config                    = ensure_config(CONFIG_PATH)   # Load config again for on the fly changes

inductex_ixi_config               = quantum_config["simulations"]["inductex"]["ixi"]
inductex_ldf_config               = quantum_config["simulations"]["inductex"]["layer_definition_file"]

# Generate InductEx simulation GDS files for all designs
designs = [full_design, full_no_jj_design, junction_design, transmon_design, transmon_no_jj_design, resonator_design, feedline_design]
for design_name, design_obj in zip(design_names, designs):
    make_inductex_cap_sim(design=design_obj, 
                          design_name=design_name, 
                          create_mask=False, 
                          mask_component=design_name, 
                          inductex_ixi_config=inductex_ixi_config, 
                          inductex_ldf_config=inductex_ldf_config
                          )

    make_inductex_ind_sim(design=design_obj, 
                          design_name=design_name, 
                          create_mask=False, 
                          mask_component=design_name, 
                          inductex_ixi_config=inductex_ixi_config, 
                          inductex_ldf_config=inductex_ldf_config
                          )

    make_inductex_sparam_sim(design=design_obj, 
                             design_name=design_name, 
                             create_mask=False, 
                             mask_component=design_name, 
                             inductex_ixi_config=inductex_ixi_config, 
                             inductex_ldf_config=inductex_ldf_config
                             )


[config] Validation successful: 'quantum_config.json' is valid.
[INFO] simulations"]["inductex"]["ixi"]["sparams_steps is: 8 (Default: 200)
[INFO] simulations"]["palace"]["eigenmode"]["fine_mesh_min_size_components is: 5e-06 (Default: 1e-05)
[INFO] simulations"]["palace"]["eigenmode"]["fine_mesh_min_size_junction is: 1e-05 (Default: 2e-06)


## Setup PALACE Mesh Simulation Files 

In [11]:
if run_palace_setup:
    quantum_config                               = ensure_config(CONFIG_PATH)   # Load config here for easy use

    palace_cap_config               = quantum_config["simulations"]["palace"]["capacitance"]
    palace_eigenmode_config         = quantum_config["simulations"]["palace"]["eigenmode"]
    palace_eigenmode_config["lj"]   = lj

    cap_sim_list = []
    cap_sim_list.append(make_palace_cap_sim(transmon_design, "transmon", palace_cap_config))        
    cap_sim_list.append(make_palace_cap_sim(transmon_no_jj_design, "transmon_no_jj", palace_cap_config))
    cap_sim_list.append(make_palace_cap_sim(resonator_design, "resonator", palace_cap_config))        
    cap_sim_list.append(make_palace_cap_sim(feedline_design, "feedline", palace_cap_config))          

    eigen_sim_list = []
    eigen_sim_list.append(make_palace_eigenmode_sim(full_design, "full", palace_eigenmode_config))   
    eigen_sim_list.append(make_palace_eigenmode_sim(full_no_jj_design, "full_no_jj", palace_eigenmode_config))    
    eigen_sim_list.append(make_palace_eigenmode_sim(resonator_design, "resonator", palace_eigenmode_config))


In [12]:
# UNIMPLEMENTED PALACE MESH SIM FILES

#make_palace_cap_sim_full_gds(full_design, qubit_geo)
#make_palace_cap_sim_full_no_jj_gds(full_no_jj_design, qubit_geo)
#make_palace_cap_sim_junction_gds(junction_design, qubit_geo)

#make_palace_eigenmode_sim(junction_design, "junction", palace_eigenmode_config)
#make_palace_eigenmode_sim(feedline_design, "feedline", palace_eigenmode_config)
#make_palace_eigenmode_sim(transmon_design, "transmon", palace_eigenmode_config)   
#make_palace_eigenmode_sim(transmon_no_jj_design, "transmon_no_jj", palace_eigenmode_config)

## Optionally show sims and their conductor indices

In [13]:
if palace_show_conductor_indices:
    for cap_sim in cap_sim_list:
        cap_sim.display_conductor_indices()
    for eigen_sim in eigen_sim_list:
        epr_data = eigen_sim.retrieve_EPR_data()  
        mode_port_epr = eigen_sim.retrieve_mode_port_EPR()

## Run Inductex simulations

In [14]:
# Select either capacitance, inductance, sparams, or eigenmode 
if run_inductex:
    run_inductex_simulation("transmon", "capacitance", keep_open=False)
    run_inductex_simulation("transmon_no_jj", "capacitance", keep_open=False)
    run_inductex_simulation("resonator", "capacitance", keep_open=False)
    run_inductex_simulation("feedline", "capacitance", keep_open=False)        

    run_inductex_simulation("resonator", "inductance", keep_open=False)

    run_inductex_simulation("transmon", "sparams", keep_open=False)
    run_inductex_simulation("transmon_no_jj", "sparams", keep_open=False)
    run_inductex_simulation("resonator", "sparams", keep_open=False)

## Run PALACE Simulations

In [15]:
if run_palace:
    run_palace_simulation("transmon", "capacitance", palace_eigenmode_config["number_of_cores"], keep_open=False)
    run_palace_simulation("transmon_no_jj", "capacitance", palace_eigenmode_config["number_of_cores"], keep_open=False)
    run_palace_simulation("resonator", "capacitance", palace_eigenmode_config["number_of_cores"], keep_open=False)
    run_palace_simulation("feedline", "capacitance", palace_eigenmode_config["number_of_cores"], keep_open=False)

    run_palace_simulation("full", "eigenmode", palace_eigenmode_config["number_of_cores"], keep_open=False)
    run_palace_simulation("full_no_jj", "eigenmode", palace_eigenmode_config["number_of_cores"], keep_open=False)
    run_palace_simulation("resonator", "eigenmode", palace_eigenmode_config["number_of_cores"], keep_open=False)


## Mess around code below

In [ ]:
quantum_config                  = ensure_config(CONFIG_PATH)
palace_cap_config               = quantum_config["simulations"]["palace"]["capacitance"]
palace_eigenmode_config         = quantum_config["simulations"]["palace"]["eigenmode"]
palace_eigenmode_config["lj"]   = lj

In [ ]:
make_palace_cap_sim(resonator_design, "resonator", palace_cap_config)
run_palace_simulation("resonator", "capacitance", palace_cap_config["number_of_cores"], keep_open=False)

In [ ]:
make_palace_eigenmode_sim(full_design, "full", palace_eigenmode_config)
run_palace_simulation("full", "eigenmode", palace_eigenmode_config["number_of_cores"], keep_open=False)

In [ ]:
make_palace_eigenmode_sim(resonator_design, "resonator", palace_eigenmode_config)
run_palace_simulation("resonator", "eigenmode", palace_eigenmode_config["number_of_cores"], keep_open=False)

In [ ]:
if False:
    from qiskit_metal import designs, Dict
    x = designs.DesignPlanar({}, overwrite_enabled=True);
    gui = MetalGUI(transmon_design);
    gui.main_window.load_stylesheet_dark();
    gui.main_window.showMaximized();
    gui.autoscale();
    gui.qApp.exec_();
    #design.rebuild()
    #gui.rebuild()
    #gui.autoscale()
    #gui.show()  # Display the design in the GUI

    gui.main_window.force_close = True  
    gui.main_window.close();
    gui.qApp.quit()

In [ ]:
from SQDMetal.Utilities.CpwParams import CpwParams  
from SQDMetal.Utilities.QUtilities import QUtilities 
from helpers import extract_um

component = full_design.components["feedline"]

trace_width = component.options['trace_width']
trace_gap = component.options['trace_gap']

# Extract trace width and gap from your feedline  
trace_width = QUtilities.parse_value_length(trace_width)  
trace_gap = QUtilities.parse_value_length(trace_gap)  
  
# Get substrate parameters from design  
cpw_params = CpwParams.fromQDesign(design=full_design, chip_name='main')  
# Calculate impedance  
impedance = CpwParams.calc_impedance(  
    tr_wid=trace_width,  
    tr_gap=trace_gap,  
    er=cpw_params.rel_permittivity,  
    h=cpw_params.dielectric_thickness  
)  
print(f"Feedline impedance: {impedance:.2f} Ω")
  

In [16]:
results, time = collect_inductex_results(design_names, inductex_simulation_types)

save_results_to_json(results, name="data/inductex_sim_results", save=True, orient="records", indent=2)
save_results_to_json(time, name="data/inductex_sim_times", save=True, orient="records", indent=2)

✅ Saved to data/inductex_sim_results.json
✅ Saved to data/inductex_sim_times.json


In [21]:
results, time = collect_palace_results(design_names, palace_simulation_types)

save_results_to_json(results, name="data/palace_sim_results", save=True, orient="records", indent=2)
save_results_to_json(time, name="data/palace_sim_times", save=True, orient="records", indent=2)

✅ Saved to data/palace_sim_results.json
✅ Saved to data/palace_sim_times.json
